# Launch TMDB Distributed Download Fleet

```
https://github.com/hudsonmendes/lambda-tmdb-distributed-downloader
MENDES, Hudson
14th May, 2020
London, UK
```

## Summary

The `lambda TMDB distributed downloader` is an **[AWS Lambda Function](https://aws.amazon.com/lambda/)** hooked to an **[AWS SQS Queue](https://aws.amazon.com/sqs/)**.

In order to launch our download fleet, we must then send messages to **SQS** and let our lambda function find them and start the download process.

This notebook reads the **[IMDB Titles Dataset]** in order to determine the download partitions (based in `year` and `initial` letter of the title), and sends messages for
each of those partitions.

## Environment

### Settings

In [1]:
lambda_name          = 'hudsonmendes-tmdb-downloader-lambda'
queue_name           = 'hudsonmendes-tmdb-downloader-queue'
datalake_bucket_name = 'hudsonmendes-datalake'
min_year             = 2000 

### Dependencies

In [2]:
%%bash
pip install -U pandas
pip install -r requirements.txt

Requirement already up-to-date: pandas in /Users/hudsonmendes/.pyenv/versions/3.6.10/lib/python3.6/site-packages (1.0.3)


### `tdd` deployment

We first must deploy/prepare our AWS infra-structure, and we do so by using our `cli`

In [3]:
!python tdd deploy \
    --lambda_name $lambda_name \
    --queue_name $queue_name \
    --datalake_bucket_name $datalake_bucket_name

Deploy, cleaning up environment
Deploy, saving dependencies locally
ERROR: awscli 1.18.46 has requirement botocore==1.15.46, but you'll have botocore 1.16.11 which is incompatible.
Deploy, adding code to ZIP
Deploy, ensure queue hudsonmendes-tmdb-downloader-queue
Deploy, ensure lambda hudsonmendes-tmdb-downloader-lambda
Deploy, link lambda hudsonmendes-tmdb-downloader-lambda to queue hudsonmendes-tmdb-downloader-queue
Deploy, cleaning up environment


### Imports

In [4]:
import os
import boto3
import json
import pandas as pd
from urllib.request import urlopen
from tqdm.notebook import tqdm
from tdd.pipeline import IMDbMovie

### Storage

In [5]:
IMDB_MOVIES_URL  = 'https://datasets.imdbws.com/title.basics.tsv.gz'
IMDB_MOVIES_PATH = '/tmp/title.basics.tsv.gz'

In [6]:
if not os.path.isfile(IMDB_MOVIES_PATH):
    with urlopen(IMDB_MOVIES_URL) as res:
        with open(IMDB_MOVIES_PATH, 'wb+') as out_file:
            out_file.write(res.read())
IMDB_MOVIES_PATH

'/tmp/title.basics.tsv.gz'

### AWS SQS

In [7]:
sqs = boto3.resource('sqs')
queue = sqs.get_queue_by_name(QueueName=queue_name)

## Data

### IMDB, Titles

In [8]:
df = pd.read_csv(IMDB_MOVIES_PATH, delimiter='\t', header=0)

/Users/hudsonmendes/.pyenv/versions/3.6.10/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
df = df[df.titleType == 'movie']

In [10]:
df['initial'] = df['primaryTitle'].map(IMDbMovie.get_initial_from)

In [11]:
df['year'] = df['startYear'].map(IMDbMovie.get_year_from)

### Load Analysis

In [28]:
pd_group = pd.DataFrame(df
             .groupby(by=['year', 'initial'])['tconst']
             .count()
             .sort_values(ascending=False))

In [35]:
pd_group[pd_group.tconst > 700]

,,tconst
year,initial,
2017.0,TH,2214
2016.0,TH,2135
2018.0,TH,2131
2015.0,TH,1986
2019.0,TH,1921
2014.0,TH,1910
2013.0,TH,1763
2012.0,TH,1652
2011.0,TH,1573


### Fleet Partitions

In [12]:
import math, datetime
partitions = set()
for year, initial in tqdm(df[['year', 'initial']].values):
    if year and \
            not math.isnan(year) and \
            year >= min_year and \
            year <= datetime.datetime.now().year:
        year = int(year)
        initial = initial.replace(' ', '_')
        partitions.add((year, initial))
partitions = sorted(partitions)

In [13]:
len(partitions)

8698

## Launching

In [14]:
partition_iterator = iter(partitions)

### First Batch

Here we schedule just a few messages, to see if everything is alright.

In [15]:
messages_max = 3
messages_count = 0
for year, initial in partition_iterator:
    print((year, initial))
    message = { 'year': year, 'initial': initial }
    body = json.dumps(message)
    queue.send_message(MessageBody=body)
    messages_count += 1
    if messages_count >= messages_max:
        break

(2000, '01')
(2000, '10')
(2000, '11')


### Everything

In [16]:
for year, initial in partition_iterator:
    print((year, initial))
    message = { 'year': year, 'initial': initial }
    body = json.dumps(message)
    queue.send_message(MessageBody=body)

(2000, '12')
(2000, '13')
(2000, '15')
(2000, '17')
(2000, '18')
(2000, '19')
(2000, '1_')
(2000, '20')
(2000, '21')
(2000, '23')
(2000, '24')
(2000, '27')
(2000, '28')
(2000, '2_')
(2000, '30')
(2000, '33')
(2000, '36')
(2000, '3R')
(2000, '3_')
(2000, '42')
(2000, '45')
(2000, '4_')
(2000, '60')
(2000, '66')
(2000, '6_')
(2000, '75')
(2000, '76')
(2000, '7T')
(2000, '80')
(2000, '8O')
(2000, '8_')
(2000, '90')
(2000, '94')
(2000, '99')
(2000, '9B')
(2000, 'AA')
(2000, 'AB')
(2000, 'AC')
(2000, 'AD')
(2000, 'AF')
(2000, 'AG')
(2000, 'AH')
(2000, 'AI')
(2000, 'AJ')
(2000, 'AK')
(2000, 'AL')
(2000, 'AM')
(2000, 'AN')
(2000, 'AP')
(2000, 'AQ')
(2000, 'AR')
(2000, 'AS')
(2000, 'AT')
(2000, 'AU')
(2000, 'AV')
(2000, 'AY')
(2000, 'AZ')
(2000, 'A_')
(2000, 'BA')
(2000, 'BE')
(2000, 'BG')
(2000, 'BH')
(2000, 'BI')
(2000, 'BK')
(2000, 'BL')
(2000, 'BM')
(2000, 'BO')
(2000, 'BR')
(2000, 'BS')
(2000, 'BU')
(2000, 'BY')
(2000, 'CA')
(2000, 'CE')
(2000, 'CF')
(2000, 'CH')
(2000, 'CI')
(2000, 'CL')

(2001, 'YU')
(2001, 'YV')
(2001, 'Y_')
(2001, 'ZA')
(2001, 'ZB')
(2001, 'ZE')
(2001, 'ZH')
(2001, 'ZI')
(2001, 'ZO')
(2001, 'ZS')
(2001, 'ZU')
(2001, 'Z_')
(2002, '10')
(2002, '12')
(2002, '13')
(2002, '15')
(2002, '16')
(2002, '17')
(2002, '18')
(2002, '19')
(2002, '1_')
(2002, '20')
(2002, '21')
(2002, '23')
(2002, '24')
(2002, '25')
(2002, '28')
(2002, '29')
(2002, '2W')
(2002, '2_')
(2002, '33')
(2002, '35')
(2002, '36')
(2002, '3R')
(2002, '3_')
(2002, '40')
(2002, '49')
(2002, '4_')
(2002, '50')
(2002, '5_')
(2002, '60')
(2002, '64')
(2002, '66')
(2002, '6_')
(2002, '75')
(2002, '7T')
(2002, '7_')
(2002, '80')
(2002, '86')
(2002, '8_')
(2002, '97')
(2002, '99')
(2002, '9_')
(2002, 'AA')
(2002, 'AB')
(2002, 'AC')
(2002, 'AD')
(2002, 'AF')
(2002, 'AG')
(2002, 'AH')
(2002, 'AI')
(2002, 'AJ')
(2002, 'AK')
(2002, 'AL')
(2002, 'AM')
(2002, 'AN')
(2002, 'AO')
(2002, 'AP')
(2002, 'AR')
(2002, 'AS')
(2002, 'AT')
(2002, 'AU')
(2002, 'AV')
(2002, 'AW')
(2002, 'AY')
(2002, 'AZ')
(2002, 'A_')

(2003, 'TO')
(2003, 'TR')
(2003, 'TS')
(2003, 'TU')
(2003, 'TV')
(2003, 'TW')
(2003, 'TY')
(2003, 'UD')
(2003, 'UF')
(2003, 'UH')
(2003, 'UJ')
(2003, 'UL')
(2003, 'UM')
(2003, 'UN')
(2003, 'UP')
(2003, 'UR')
(2003, 'US')
(2003, 'UT')
(2003, 'U_')
(2003, 'VA')
(2003, 'VE')
(2003, 'VF')
(2003, 'VI')
(2003, 'VL')
(2003, 'VO')
(2003, 'VZ')
(2003, 'V_')
(2003, 'WA')
(2003, 'WE')
(2003, 'WH')
(2003, 'WI')
(2003, 'WO')
(2003, 'WR')
(2003, 'WW')
(2003, 'X2')
(2003, 'XC')
(2003, 'XE')
(2003, 'XS')
(2003, 'XT')
(2003, 'XU')
(2003, 'XV')
(2003, 'YA')
(2003, 'YE')
(2003, 'YI')
(2003, 'YO')
(2003, 'YU')
(2003, 'Y_')
(2003, 'ZA')
(2003, 'ZE')
(2003, 'ZH')
(2003, 'ZI')
(2003, 'ZO')
(2003, 'ZU')
(2004, '06')
(2004, '10')
(2004, '11')
(2004, '13')
(2004, '14')
(2004, '16')
(2004, '18')
(2004, '19')
(2004, '1F')
(2004, '1N')
(2004, '1S')
(2004, '1_')
(2004, '20')
(2004, '21')
(2004, '24')
(2004, '25')
(2004, '29')
(2004, '2B')
(2004, '2_')
(2004, '30')
(2004, '36')
(2004, '3I')
(2004, '3_')
(2004, '41')

(2005, 'N_')
(2005, 'OB')
(2005, 'OC')
(2005, 'OD')
(2005, 'OF')
(2005, 'OH')
(2005, 'OJ')
(2005, 'OK')
(2005, 'OL')
(2005, 'OM')
(2005, 'ON')
(2005, 'OO')
(2005, 'OP')
(2005, 'OR')
(2005, 'OS')
(2005, 'OT')
(2005, 'OU')
(2005, 'OV')
(2005, 'OX')
(2005, 'OY')
(2005, 'O_')
(2005, 'PA')
(2005, 'PE')
(2005, 'PH')
(2005, 'PI')
(2005, 'PL')
(2005, 'PO')
(2005, 'PR')
(2005, 'PS')
(2005, 'PT')
(2005, 'PU')
(2005, 'PY')
(2005, 'P_')
(2005, 'QA')
(2005, 'QI')
(2005, 'QU')
(2005, 'R2')
(2005, 'R3')
(2005, 'RA')
(2005, 'RE')
(2005, 'RG')
(2005, 'RH')
(2005, 'RI')
(2005, 'RO')
(2005, 'RU')
(2005, 'RX')
(2005, 'RY')
(2005, 'SA')
(2005, 'SC')
(2005, 'SE')
(2005, 'SH')
(2005, 'SI')
(2005, 'SK')
(2005, 'SL')
(2005, 'SM')
(2005, 'SN')
(2005, 'SO')
(2005, 'SP')
(2005, 'SQ')
(2005, 'SR')
(2005, 'SS')
(2005, 'ST')
(2005, 'SU')
(2005, 'SV')
(2005, 'SW')
(2005, 'SY')
(2005, 'SZ')
(2005, 'TA')
(2005, 'TB')
(2005, 'TC')
(2005, 'TE')
(2005, 'TH')
(2005, 'TI')
(2005, 'TK')
(2005, 'TO')
(2005, 'TR')
(2005, 'TS')

(2007, 'EQ')
(2007, 'ER')
(2007, 'ES')
(2007, 'ET')
(2007, 'EU')
(2007, 'EV')
(2007, 'EX')
(2007, 'EY')
(2007, 'EZ')
(2007, 'E_')
(2007, 'F9')
(2007, 'FA')
(2007, 'FB')
(2007, 'FE')
(2007, 'FI')
(2007, 'FL')
(2007, 'FM')
(2007, 'FO')
(2007, 'FR')
(2007, 'FU')
(2007, 'FY')
(2007, 'F_')
(2007, 'GA')
(2007, 'GE')
(2007, 'GG')
(2007, 'GH')
(2007, 'GI')
(2007, 'GL')
(2007, 'GO')
(2007, 'GP')
(2007, 'GR')
(2007, 'GU')
(2007, 'GY')
(2007, 'HA')
(2007, 'HE')
(2007, 'HI')
(2007, 'HL')
(2007, 'HO')
(2007, 'HP')
(2007, 'HU')
(2007, 'HV')
(2007, 'HW')
(2007, 'HY')
(2007, 'IB')
(2007, 'IC')
(2007, 'ID')
(2007, 'IF')
(2007, 'IL')
(2007, 'IM')
(2007, 'IN')
(2007, 'IO')
(2007, 'IR')
(2007, 'IS')
(2007, 'IT')
(2007, 'IV')
(2007, 'IZ')
(2007, 'I_')
(2007, 'JA')
(2007, 'JC')
(2007, 'JE')
(2007, 'JF')
(2007, 'JH')
(2007, 'JI')
(2007, 'JJ')
(2007, 'JO')
(2007, 'JU')
(2007, 'K3')
(2007, 'KA')
(2007, 'KC')
(2007, 'KD')
(2007, 'KE')
(2007, 'KH')
(2007, 'KI')
(2007, 'KL')
(2007, 'KM')
(2007, 'KN')
(2007, 'KO')

...

(2014, 'LY')
(2014, 'L_')
(2014, 'M3')
(2014, 'M4')
(2014, 'MA')
(2014, 'MB')
(2014, 'MC')
(2014, 'MD')
(2014, 'ME')
(2014, 'MG')
(2014, 'MH')
(2014, 'MI')
(2014, 'ML')
(2014, 'MM')
(2014, 'MO')
(2014, 'MR')
(2014, 'MS')
(2014, 'MT')
(2014, 'MU')
(2014, 'MY')
(2014, 'M_')
(2014, 'NA')
(2014, 'NC')
(2014, 'ND')
(2014, 'NE')
(2014, 'NF')
(2014, 'NG')
(2014, 'NI')
(2014, 'NJ')
(2014, 'NK')
(2014, 'NN')
(2014, 'NO')
(2014, 'NS')
(2014, 'NT')
(2014, 'NU')
(2014, 'NY')
(2014, 'N_')
(2014, 'O2')
(2014, 'OA')
(2014, 'OB')
(2014, 'OC')
(2014, 'OD')
(2014, 'OF')
(2014, 'OG')
(2014, 'OH')
(2014, 'OI')
(2014, 'OJ')
(2014, 'OK')
(2014, 'OL')
(2014, 'OM')
(2014, 'ON')
(2014, 'OO')
(2014, 'OP')
(2014, 'OR')
(2014, 'OS')
(2014, 'OT')
(2014, 'OU')
(2014, 'OV')
(2014, 'OW')
(2014, 'OX')
(2014, 'OZ')
(2014, 'O_')
(2014, 'P5')
(2014, 'PA')
(2014, 'PB')
(2014, 'PE')
(2014, 'PF')
(2014, 'PH')
(2014, 'PI')
(2014, 'PK')
(2014, 'PL')
(2014, 'PM')
(2014, 'PO')
(2014, 'PR')
(2014, 'PS')
(2014, 'PU')
(2014, 'PY')

(2015, 'TY')
(2015, 'TZ')
(2015, 'T_')
(2015, 'UB')
(2015, 'UC')
(2015, 'UD')
(2015, 'UE')
(2015, 'UF')
(2015, 'UG')
(2015, 'UI')
(2015, 'UK')
(2015, 'UL')
(2015, 'UM')
(2015, 'UN')
(2015, 'UO')
(2015, 'UP')
(2015, 'UR')
(2015, 'US')
(2015, 'UT')
(2015, 'UV')
(2015, 'UW')
(2015, 'UZ')
(2015, 'V8')
(2015, 'VA')
(2015, 'VE')
(2015, 'VI')
(2015, 'VL')
(2015, 'VO')
(2015, 'VR')
(2015, 'VT')
(2015, 'VU')
(2015, 'VW')
(2015, 'V_')
(2015, 'WA')
(2015, 'WE')
(2015, 'WH')
(2015, 'WI')
(2015, 'WK')
(2015, 'WO')
(2015, 'WR')
(2015, 'WU')
(2015, 'WW')
(2015, 'WY')
(2015, 'W_')
(2015, 'XC')
(2015, 'XE')
(2015, 'XI')
(2015, 'XU')
(2015, 'XX')
(2015, 'X_')
(2015, 'YA')
(2015, 'YE')
(2015, 'YI')
(2015, 'YO')
(2015, 'YP')
(2015, 'YU')
(2015, 'Y_')
(2015, 'ZA')
(2015, 'ZB')
(2015, 'ZE')
(2015, 'ZH')
(2015, 'ZI')
(2015, 'ZK')
(2015, 'ZL')
(2015, 'ZN')
(2015, 'ZO')
(2015, 'ZP')
(2015, 'ZS')
(2015, 'ZU')
(2015, 'ZY')
(2015, 'Z_')
(2016, '03')
(2016, '04')
(2016, '07')
(2016, '10')
(2016, '11')
(2016, '12')

(2017, 'AP')
(2017, 'AQ')
(2017, 'AR')
(2017, 'AS')
(2017, 'AT')
(2017, 'AU')
(2017, 'AV')
(2017, 'AW')
(2017, 'AX')
(2017, 'AY')
(2017, 'AZ')
(2017, 'A_')
(2017, 'BA')
(2017, 'BB')
(2017, 'BE')
(2017, 'BF')
(2017, 'BG')
(2017, 'BH')
(2017, 'BI')
(2017, 'BK')
(2017, 'BL')
(2017, 'BM')
(2017, 'BN')
(2017, 'BO')
(2017, 'BP')
(2017, 'BR')
(2017, 'BU')
(2017, 'BY')
(2017, 'B_')
(2017, 'CA')
(2017, 'CC')
(2017, 'CE')
(2017, 'CF')
(2017, 'CH')
(2017, 'CI')
(2017, 'CL')
(2017, 'CO')
(2017, 'CR')
(2017, 'CS')
(2017, 'CT')
(2017, 'CU')
(2017, 'CY')
(2017, 'C_')
(2017, 'D5')
(2017, 'DA')
(2017, 'DC')
(2017, 'DE')
(2017, 'DH')
(2017, 'DI')
(2017, 'DJ')
(2017, 'DL')
(2017, 'DN')
(2017, 'DO')
(2017, 'DP')
(2017, 'DR')
(2017, 'DU')
(2017, 'DV')
(2017, 'DW')
(2017, 'DY')
(2017, 'DZ')
(2017, 'E1')
(2017, 'E6')
(2017, 'EA')
(2017, 'EB')
(2017, 'EC')
(2017, 'ED')
(2017, 'EE')
(2017, 'EF')
(2017, 'EG')
(2017, 'EI')
(2017, 'EK')
(2017, 'EL')
(2017, 'EM')
(2017, 'EN')
(2017, 'EO')
(2017, 'EP')
(2017, 'EQ')

(2018, 'JT')
(2018, 'JU')
(2018, 'JV')
(2018, 'KA')
(2018, 'KC')
(2018, 'KD')
(2018, 'KE')
(2018, 'KG')
(2018, 'KH')
(2018, 'KI')
(2018, 'KK')
(2018, 'KL')
(2018, 'KN')
(2018, 'KO')
(2018, 'KR')
(2018, 'KS')
(2018, 'KU')
(2018, 'KV')
(2018, 'KY')
(2018, 'LA')
(2018, 'LE')
(2018, 'LG')
(2018, 'LH')
(2018, 'LI')
(2018, 'LL')
(2018, 'LO')
(2018, 'LU')
(2018, 'LY')
(2018, 'L_')
(2018, 'MA')
(2018, 'MB')
(2018, 'MC')
(2018, 'ME')
(2018, 'MG')
(2018, 'MI')
(2018, 'ML')
(2018, 'MM')
(2018, 'MN')
(2018, 'MO')
(2018, 'MR')
(2018, 'MS')
(2018, 'MU')
(2018, 'MV')
(2018, 'MY')
(2018, 'M_')
(2018, 'NA')
(2018, 'ND')
(2018, 'NE')
(2018, 'NF')
(2018, 'NI')
(2018, 'NJ')
(2018, 'NO')
(2018, 'NS')
(2018, 'NU')
(2018, 'NY')
(2018, 'N_')
(2018, 'OA')
(2018, 'OB')
(2018, 'OC')
(2018, 'OD')
(2018, 'OF')
(2018, 'OG')
(2018, 'OH')
(2018, 'OI')
(2018, 'OJ')
(2018, 'OK')
(2018, 'OL')
(2018, 'OM')
(2018, 'ON')
(2018, 'OO')
(2018, 'OP')
(2018, 'OR')
(2018, 'OS')
(2018, 'OT')
(2018, 'OU')
(2018, 'OV')
(2018, 'OW')

(2019, 'UK')
(2019, 'UL')
(2019, 'UM')
(2019, 'UN')
(2019, 'UP')
(2019, 'UR')
(2019, 'US')
(2019, 'UT')
(2019, 'UY')
(2019, 'UZ')
(2019, 'U_')
(2019, 'V1')
(2019, 'V6')
(2019, 'VA')
(2019, 'VE')
(2019, 'VF')
(2019, 'VH')
(2019, 'VI')
(2019, 'VL')
(2019, 'VO')
(2019, 'VR')
(2019, 'VS')
(2019, 'VU')
(2019, 'VY')
(2019, 'V_')
(2019, 'WA')
(2019, 'WE')
(2019, 'WH')
(2019, 'WI')
(2019, 'WO')
(2019, 'WR')
(2019, 'WU')
(2019, 'WW')
(2019, 'WY')
(2019, 'W_')
(2019, 'XB')
(2019, 'XE')
(2019, 'XI')
(2019, 'XM')
(2019, 'XS')
(2019, 'XX')
(2019, 'XY')
(2019, 'X_')
(2019, 'YA')
(2019, 'YE')
(2019, 'YI')
(2019, 'YO')
(2019, 'YS')
(2019, 'YU')
(2019, 'YV')
(2019, 'Y_')
(2019, 'ZA')
(2019, 'ZE')
(2019, 'ZH')
(2019, 'ZI')
(2019, 'ZL')
(2019, 'ZN')
(2019, 'ZO')
(2019, 'ZR')
(2019, 'ZU')
(2019, 'ZW')
(2019, 'ZZ')
(2019, 'Z_')
(2020, '00')
(2020, '09')
(2020, '0_')
(2020, '10')
(2020, '11')
(2020, '12')
(2020, '13')
(2020, '15')
(2020, '16')
(2020, '18')
(2020, '19')
(2020, '1I')
(2020, '1_')
(2020, '20')

### Evaluation

In [17]:
df[df['tconst'] == 'tt9686708']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,initial,year
6699377,tt9686708,movie,The King of Staten Island,The King of Staten Island,0,2020,\N,136,"Comedy,Drama",TH,2020.0
